In [21]:
import requests
from io import BytesIO
from openai import OpenAI
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display

client = OpenAI()

In [4]:
# constants
VECTOR_STORE_ID="vs_6815ebab19a481918885f3d28b9c887a"

In [5]:
# list files
result = client.vector_stores.files.list(
    vector_store_id= VECTOR_STORE_ID
)
print(result)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-4yNKpeK2KgqGf1kfV6uGvg', created_at=1746267142, last_error=None, object='vector_store.file', status='completed', usage_bytes=1978942, vector_store_id='vs_6815ebab19a481918885f3d28b9c887a', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=500, max_chunk_size_tokens=1000), type='static'))], has_more=False, object='list', first_id='file-4yNKpeK2KgqGf1kfV6uGvg', last_id='file-4yNKpeK2KgqGf1kfV6uGvg')


In [6]:
# query files
user_query = "que articulo habla sobre el oficial de compra?"

results = client.vector_stores.search(
    vector_store_id=VECTOR_STORE_ID,
    query=user_query,
)
# print(json.dumps(results.to_dict(), ensure_ascii=False, separators=(',', ':')))
for result in results.data:
    print(f"Archivo: {getattr(result, 'filename', None)}")
    print(f"ID de archivo: {getattr(result, 'file_id', None)}")
    print(f"Puntaje: {getattr(result, 'score', None)}")
    # Mostrar el contenido (puede ser una lista de objetos Content)
    for content in getattr(result, 'content', []):
        print(f"Contenido: {getattr(content, 'text', '')}")
    print("-" * 40)

Archivo: LeyGeneralDeContratacionesPublicas.pdf
ID de archivo: file-4yNKpeK2KgqGf1kfV6uGvg
Puntaje: 0.8383191630708016
Contenido: OTRAS MODALIDADES DIFERENCIADAS DE PROCEDIMIENTOS DE 
SELECCIÓN 

SUBCAPÍTULO 1 
Subasta inversa electrónica 

Artículo 96. Evaluación de ofertas en subasta inversa electrónica 

96.1. Mediante subasta inversa electrónica se contratan bienes y servicios 
comunes que cuenten con ficha técnica. La utilización del procedimiento de selección 
puede ser opcional u obligatoria conforme lo determine Perú Compras en el 
documento que aprueba la referida ficha. 

96.2. En el requerimiento de bienes y servicios comunes con ficha técnica, no se 
pueden incluir requisitos de calificación adicionales o diferentes a aquellos 
contemplados en la ficha técnica o en los documentos de información 
complementaria, salvo que la normativa específica que regula el objeto de la 
convocatoria exija algún requisito obligatorio. 

96.3. En la subasta inversa electrónica no se realiza

In [27]:
PROMPT = """
Asiste a los usuarios en encontrar información específica en documentos legales proporcionados. Cuando un usuario formule una pregunta, revisa los documentos para identificar información relevante que pueda responder a la consulta.

- Si encuentras la información requerida en los documentos, proporciona una respuesta clara y precisa.
- Si no puedes encontrar la información necesaria o no estás seguro, responde con "No tengo información sobre eso."

# Steps

1. Lee cuidadosamente la pregunta del usuario.
2. Identifica las palabras clave y la intención de la pregunta.
3. Revisa los documentos en busca de secciones relacionadas con la pregunta.
4. Analiza el texto para encontrar información relevante que pueda responder a la pregunta del usuario.
5. Si encuentras la información necesaria, formula una respuesta clara y precisa.
6. Si no puedes encontrar la información, responde con "No tengo información sobre eso."

# Output Format

- Proporcionar respuestas claras y concisas en forma de párrafo.
- Utilizar markdown para resaltar información clave si es necesario.
- Incluir citas o referencias a secciones específicas del documento si es posible.
- Si no se encuentra la información, usar la respuesta: "No tengo información sobre eso."

# Examples

**Ejemplo 1:**

**Pregunta del usuario:** ¿Qué dice la cláusula de confidencialidad en el contrato?

**Proceso:**
- Busca "cláusula de confidencialidad" en los documentos.
- Lee los párrafos que contienen información relevante sobre la cláusula de confidencialidad.
- Determina si hay detalles sobre las obligaciones de las partes involucradas.

**Respuesta:** La cláusula de confidencialidad establece que ambas partes están obligadas a no divulgar ninguna información obtenida durante el transcurso del contrato, salvo lo permitido por ley. (Este ejemplo es breve para ilustración; las respuestas reales deben estar más detalladas y con lenguaje legal preciso si el documento lo permite.)

**Ejemplo 2:**

**Pregunta del usuario:** ¿Cuáles son los términos de renovación del contrato?

**Proceso:**
- Busca "términos de renovación" o "renovación del contrato" en los documentos.
- Lee las secciones relevantes para identificar los términos.

**Respuesta:** No tengo información sobre eso. (Utiliza esta respuesta si tras revisar el documento no encuentras la información requerida.)

# Notes

- Asegúrate de leer los documentos con atención para no pasar por alto detalles clave.
- La precisión en la interpretación legal es crucial; utiliza el lenguaje legal cuando sea necesario.
- Recuerda que si no estás completamente seguro sobre la información, es mejor indicar que no tienes información sobre eso para mantener la autoridad y credibilidad.
- You are a lawyer and you are answering a question about a legal document
- Si no puedes encontrar la respuesta, di "No tengo información sobre eso"""""
MODEL="gpt-4.1-mini"

In [31]:
# LLM
response = client.responses.create(
    model=MODEL,
    instructions=PROMPT,
    temperature=0.5,
    input="que articulo habla sobre el oficial de compra?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [VECTOR_STORE_ID],
        "max_num_results": 10
    }],
    include=["file_search_call.results"]
)
print(json.dumps(response.to_dict(), ensure_ascii=False, separators=(',', ':')))
for output in response.output:
    if hasattr(output, "content"):
        for content in getattr(output, "content", []):
            if hasattr(content, "text"):
                print(f"Respuesta: {content.text}")
    elif hasattr(output, "text"):
        print(f"Respuesta: {output.text}")
print("-" * 40)


{"id":"resp_681601f5cae881919c853926a24844d307ad1592d7f31b03","created_at":1746272757.0,"error":null,"incomplete_details":null,"instructions":"\nAsiste a los usuarios en encontrar información específica en documentos legales proporcionados. Cuando un usuario formule una pregunta, revisa los documentos para identificar información relevante que pueda responder a la consulta.\n\n- Si encuentras la información requerida en los documentos, proporciona una respuesta clara y precisa.\n- Si no puedes encontrar la información necesaria o no estás seguro, responde con \"No tengo información sobre eso.\"\n\n# Steps\n\n1. Lee cuidadosamente la pregunta del usuario.\n2. Identifica las palabras clave y la intención de la pregunta.\n3. Revisa los documentos en busca de secciones relacionadas con la pregunta.\n4. Analiza el texto para encontrar información relevante que pueda responder a la pregunta del usuario.\n5. Si encuentras la información necesaria, formula una respuesta clara y precisa.\n6. Si

In [32]:
# Print the response
# print(response.output[1].content[0].text)
markdown_text = response.output[1].content[0].text
display(Markdown(markdown_text))

El artículo que habla sobre el oficial de compra es el **Artículo 58** de la Ley General de Contrataciones Públicas. En este artículo se establece que el oficial de compra es el funcionario o servidor designado por la Dirección Ejecutiva de Contrataciones (DEC), responsable de la fase de selección. Además, se indica que un oficial de compra puede ser designado en más de un procedimiento de selección simultáneamente.

También se menciona al oficial de compra en otros artículos relacionados con sus funciones específicas en los procedimientos de selección, como en los artículos 96, 97 y 98, donde se detallan sus responsabilidades en la subasta inversa electrónica y en la comparación de precios, incluyendo la revisión de requisitos de calificación, evaluación de ofertas y otorgamiento de la buena pro.

Por ejemplo, en el artículo 96.5 se señala que el oficial de compra revisa los requisitos de calificación de los postores que ocuparon los primeros lugares en la evaluación económica y otorga la buena pro al primer lugar. En el artículo 97.2, el oficial de compra registra la convocatoria y remite invitación a proveedores. En el artículo 98 se detalla que el oficial de compra verifica requisitos, evalúa ofertas económicas y otorga la buena pro mediante publicación.

En resumen, el oficial de compra está regulado principalmente en el Artículo 58, y sus funciones específicas se desarrollan en los artículos 96, 97 y 98 de la Ley General de Contrataciones Públicas    .